In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='my_api_key',
    model_name="llama-3.3-70b-versatile"
)

response= llm.invoke("The first person to land on moon was...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader= WebBaseLoader("https://www.amazon.jobs/en/jobs/2808739/software-development-engineer-internship-2025-us")
page_data= loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Software Development Engineer Internship - 2025 (US) - Job ID: 2808739 | Amazon.jobs
Skip to main contentHomeTeamsLocationsJob categoriesMy careerMy applicationsMy profileAccount securitySettingsSign outResourcesDisability accommodationsBenefitsDiversity, equity, and inclusionInterview tipsLeadership principlesWorking at AmazonFAQ×Software Development Engineer Internship - 2025 (US)Job ID: 2808739 | Amazon.com Services LLCApply nowView transcriptDESCRIPTIONAmazon internships are full-time positions, and interns should expect to work Monday-Friday, up to 40 hours per week typically between 8am-5pm. Specific team norms around working hours will be communicated by your manager. Interns should not have conflicts such as class or other employment during the Amazon work-day. Applicants should have a minimum of one quarter/semester/trimester remaining in their studies after their internship concludes. Applicants will be considered for SDE roles including the Amazon Propel Program (APP) throug

In [3]:
from langchain_core.prompts import PromptTemplate

prompt_extract= PromptTemplate.from_template(
"""
### SCRAPED TEXT FROM WEBSITE:
{page_data}
### INSTRUCTION:
The scraped text is from the career's page of a website.
Your job is to extract the job postings and return them in JSON format containing the following keys:
'role', 'experience', 'skills' and 'description'.
Only return the valid JSON.
### VALID JSON (NO PREAMBLE):
"""
)

chain_extract= prompt_extract | llm
res= chain_extract.invoke(input={'page_data': page_data})
type(res.content)

str

In [4]:
from langchain_core.output_parsers import JsonOutputParser

json_parser= JsonOutputParser()
json_res= json_parser.parse(res.content)
json_res

{'role': 'Software Development Engineer Internship',
 'experience': 'At least one quarter/semester/trimester remaining in studies, experience with modern languages such as Java, Python, C++, or C#',
 'skills': ['Java',
  'Python',
  'C++',
  'C#',
  'Object-oriented design',
  'Distributed systems',
  'Algorithms',
  'Relational databases',
  'Optimization mathematics',
  'Linear programming',
  'Nonlinear optimization'],
 'description': 'Collaborate with experienced cross-disciplinary Amazonians to conceive, design, and bring innovative products and services to market. Design and build innovative technologies in a large distributed computing environment, and help lead fundamental changes in the industry.'}

In [5]:
type(json_res)

dict

In [6]:
import pandas as pd

df= pd.read_csv("my_portfolio.csv")
df

,Techstack,Link
0,"Python, C/C++, Java, SQL, MySQL, MongoDB",https://divyanshirw.github.io/
1,"HTML, CSS, JavaScript, React.js, Django, Node....",https://divyanshirw.github.io/
2,"Machine Learning, Data Science and Analysis, N...",https://divyanshirw.github.io/


In [7]:
import uuid
import chromadb

client= chromadb.PersistentClient('vectorstore')
collection= client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                      metadatas= {"link": row["Link"]},
                      ids= [str(uuid.uuid4())])

In [11]:
job = json_res
job

{'role': 'Software Development Engineer Internship',
 'experience': 'At least one quarter/semester/trimester remaining in studies, experience with modern languages such as Java, Python, C++, or C#',
 'skills': ['Java',
  'Python',
  'C++',
  'C#',
  'Object-oriented design',
  'Distributed systems',
  'Algorithms',
  'Relational databases',
  'Optimization mathematics',
  'Linear programming',
  'Nonlinear optimization'],
 'description': 'Collaborate with experienced cross-disciplinary Amazonians to conceive, design, and bring innovative products and services to market. Design and build innovative technologies in a large distributed computing environment, and help lead fundamental changes in the industry.'}

In [15]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Divyanshi Rawat, a motivated and skilled student looking for an internship opportunity. 
    Your goal is to craft a compelling cold email to apply for the internship mentioned above, 
    highlighting your relevant skills, experiences and enthusiasm for the role.
    
    - Address the recipient professionally.
    - Clearly state your interest in the position.
    - Mention 2-3 key skills or experiences that make you a strong candidate.
    - Show enthusiasm for the company and the role.
    - Keep it concise and action-oriented, with a polite closing.

    Do not provide a preamble.
    
    ### EMAIL (NO PREAMBLE):
    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job)})
print(res.content)


Subject: Application for Software Development Engineer Internship

Dear Hiring Manager,

I am excited to express my interest in the Software Development Engineer Internship at Amazon. With a strong passion for innovative technologies and a solid foundation in computer science, I am confident that I would be a great fit for this role.

As a skilled and motivated student, I possess a strong proficiency in Java, Python, and object-oriented design, which I believe are essential skills for this position. Additionally, my experience with algorithms and distributed systems has equipped me with a solid understanding of large-scale computing environments. I am eager to apply my skills and knowledge to contribute to the development of innovative products and services at Amazon.

I am particularly drawn to Amazon's commitment to innovation and customer satisfaction, and I am impressed by the company's efforts to lead fundamental changes in the industry. I am excited about the opportunity to colla